In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, precision_recall_fscore_support
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import cross_validate, cross_val_score
from random import seed

In [2]:
df = pd.read_csv('final_NFL_dataset.csv')

In [3]:
df.head()

,Unnamed: 0,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,x,...,foulName1_Offensive Holding,foulName1_Offensive Pass Interference,passResult_R,offenseFormation_I_FORM,offenseFormation_JUMBO,offenseFormation_PISTOL,offenseFormation_SHOTGUN,offenseFormation_SINGLEBACK,foulName2_Offensive Pass Interference,playDirection_right
0,57098,2.022092e+09,3599.0,47836.0,Miles Sanders,14.0,2022-09-19 23:03:36.200000,26.0,PHI,48.84,...,0,0,0,0,0,0,1,0,0,1
1,44502,2.022092e+09,1649.0,47807.0,Josh Jacobs,67.0,2022-09-18 17:35:35.099999,28.0,LV,68.17,...,1,0,0,0,0,0,1,0,0,0
2,39265,2.022092e+09,3245.0,46094.0,Hayden Hurst,12.0,2022-09-18 18:54:15.900000,88.0,CIN,67.71,...,0,0,0,0,0,0,0,1,0,1
3,15407,2.022092e+09,3419.0,43722.0,J.D. McKissic,10.0,2022-09-18 15:46:54.400000,23.0,WAS,37.80,...,0,0,0,0,0,0,1,0,0,1
4,18225,2.022092e+09,2198.0,53430.0,Trevor Lawrence,31.0,2022-09-18 14:49:42.500000,16.0,JAX,83.37,...,0,0,0,0,0,0,1,0,0,0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 'expectedPointsAdded'), df.expectedPointsAdded, test_size = .3, random_state = 47)

In [21]:
str_list = df.select_dtypes(include = object).columns
id_train = X_train[str_list]
id_test = X_test[str_list]
X_train.drop(columns= str_list, inplace= True)
X_test.drop(columns=str_list, inplace= True)

In [22]:
RF_pipe = make_pipeline(
    SimpleImputer(strategy= 'mean'),
    StandardScaler(),
    RandomForestRegressor(random_state= 47, n_estimators = 112)
)

In [27]:
X_rav = df[df['club'] == 'BAL']
X_rav.drop(columns = 'expectedPointsAdded')
X_rav.drop(columns= str_list, inplace= True)

/var/folders/1v/cty37wrs42196x0dzbz8j4jw0000gn/T/ipykernel_61468/1383529358.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_rav.drop(columns= str_list, inplace= True)


In [28]:
y_rav = df[df['club'] == 'BAL']
y_rav = y_rav['expectedPointsAdded']

In [29]:
RF_pipe.fit(X_rav, y_rav)

/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(n_estimators=112, random_state=47))])

In [30]:
feats = ['playResult', 'prePenaltyPlayResult', 'yardsToGo', 'down']

In [31]:
def predictions(features, deltas):
    """ Changing features by deltas to see how it will
        affect the expected points added
    """
    rav2 = X_rav.copy()
    for f, d in zip((features), deltas):
        rav2[features] += deltas
    return np.mean(RF_pipe.predict(X_rav)) / np.mean(RF_pipe.predict(rav2))  

# Scenario 1:

In [46]:
playdelta = [i for i in range(10, -5, -1)]
epa_delta = [predictions(['playResult'], [delta]) for delta in playdelta]

/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warning

In [47]:
epa_delta

[0.914433547593048,
 0.9295856142316591,
 0.9312601130535585,
 0.9335897261824632,
 0.9344692792247867,
 0.9385860351529622,
 0.9471809519072142,
 0.9588065290328716,
 0.9636245821332129,
 0.9831499823945669,
 1.0,
 1.0237370746095384,
 1.0301608884605442,
 1.036385774254256,
 1.0456316132740535]

# Scenario 2:

In [44]:
scenario2_shotgun = predictions(['offenseFormation_SHOTGUN', 'playResult', 'yardsToGo', 'down'], [1,4, 6, 2])

print('Shootgun:', scenario2_shotgun)

scenario2_singleback = predictions(['offenseFormation_SINGLEBACK', 'playResult', 'yardsToGo', 'down'], [1,4, 6, 2])

print('Singleback', scenario2_singleback)


scenario2_I_FORM = predictions(['offenseFormation_I_FORM', 'playResult', 'yardsToGo', 'down'], [1,4, 6, 2])

print('I Form:',scenario2_I_FORM)


scenario2_PISTOL = predictions(['offenseFormation_PISTOL', 'playResult', 'yardsToGo', 'down'], [1,4, 6, 2])
print('Pistol:', scenario2_PISTOL)

scenario2_JUMBO = predictions(['offenseFormation_JUMBO', 'playResult', 'yardsToGo', 'down'], [1,4, 6, 2])
print('Jumbo:', scenario2_JUMBO)

Shootgun: 0.8558623425188889
Singleback 0.8519353775782085
I Form: 0.8521415473016388
Pistol: 0.8522180737490715
Jumbo: 0.8521415473016388


/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warning

# Scenario 3:

In [43]:
scenario3_shotgun =predictions(['offenseFormation_SHOTGUN','playResult', 'yardsToGo', 'down', 'prePenaltyPlayResult'], [1, 14, 10, 1, 16])
print('Shootgun:', scenario3_shotgun)

scenario3_singleback = predictions(['offenseFormation_SINGLEBACK', 'playResult', 'yardsToGo', 'down', 'prePenaltyPlayResult'], [1, 14, 10, 1, 16])
print('Singleback', scenario3_singleback)

scenario3_I_FORM = predictions(['offenseFormation_I_FORM', 'playResult', 'yardsToGo', 'down', 'prePenaltyPlayResult'], [1, 14, 10, 1, 16])
print('I Form:',scenario3_I_FORM)

scenario3_PISTOL = predictions(['offenseFormation_PISTOL', 'playResult', 'yardsToGo', 'down', 'prePenaltyPlayResult'], [1, 14, 10, 1, 16])
print('Pistol:', scenario3_PISTOL)

scenario3_JUMBO = predictions(['offenseFormation_JUMBO', 'playResult', 'yardsToGo', 'down', 'prePenaltyPlayResult'], [1, 14, 10, 1, 16])
print('Jumbo:', scenario3_JUMBO)

Shootgun: 0.6131623265558811
Singleback 0.6111441202724303
I Form: 0.6112502089786642
Pistol: 0.611289583481916
Jumbo: 0.6112502089786642


/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/cyn/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['penaltyYards']. At least one non-missing value is needed for imputation with strategy='mean'.
  warning